In [1]:
import os
import numpy as np

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend
from tensorflow import distribute
import pickle

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Flatten

In [3]:
import tensorflow as tf

# Get the list of GPUs available
gpus = tf.config.experimental.list_physical_devices('GPU')
# Enable memory growth on each GPU
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
if gpus:
    try:
        # Set memory limit for the first GPU
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])  # Limit to 1GB, for example
    except RuntimeError as e:
        print(e)

In [16]:
BATCH_SIZE = 32 
learning_rate = 0.005
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
image_size = (32, 32, 3)
sampling = 2

In [12]:
#(training_images, training_labels) , (validation_images, validation_labels) = tf.keras.datasets.cifar10.load_data()
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  input_images = tf.keras.layers.UpSampling2D(size=(sampling, sampling))(input_images)
  output_ims = preprocess_input(input_images)
  return output_ims
strategy = distribute.get_strategy()
with strategy.scope():
    fe_model = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size[0] * sampling, image_size[1] * sampling, image_size[2]))

In [13]:
def load_batch(fpath, label_key="labels"):
    """Internal utility for parsing CIFAR data.

    Args:
        fpath: path the file to parse.
        label_key: key for label data in the retrieve
            dictionary.

    Returns:
        A tuple `(data, labels)`.
    """
    with open(fpath, "rb") as f:
        d = pickle.load(f, encoding="bytes")
        # decode utf8
        d_decoded = {}
        for k, v in d.items():
            d_decoded[k.decode("utf8")] = v
        d = d_decoded
    data = d["data"]
    labels = d[label_key]

    data = data.reshape(data.shape[0], 3, 32, 32)
    return data, labels

def load_data_keras(path):
    num_train_samples = 50000
    
    x_train = np.empty((10000, 3, 32, 32), dtype="uint8")
    y_train = np.empty((num_train_samples,), dtype="uint8")
    features_train = []
    # batches are within an inner folder
    path = os.path.join(path, "cifar-10-batches-py")
    for i in range(1, 6):
        fpath = os.path.join(path, "data_batch_" + str(i))
        print(fpath)
        (
            x_train,
            y_train[(i - 1) * 10000 : i * 10000],
        ) = load_batch(fpath)
        X_train = preprocess_image_input(x_train.transpose(0, 2, 3, 1))
        print(X_train.shape)
        features_train.append(fe_model.predict(X_train, verbose=1))
        
    fpath = os.path.join(path, "test_batch")
    x_test, y_test = load_batch(fpath)
    X_test = preprocess_image_input(x_test.transpose(0, 2, 3, 1))
    features_test = fe_model.predict(X_test, verbose=1)
    
    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))
    
    return (features_train, y_train), (features_test, y_test)   

In [14]:
(features_train, training_labels) , (features_test, validation_labels) = load_data_keras("../Data")

../Data/cifar-10-batches-py/data_batch_1
(10000, 64, 64, 3)
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 68ms/step
../Data/cifar-10-batches-py/data_batch_2
(10000, 64, 64, 3)
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 69ms/step
../Data/cifar-10-batches-py/data_batch_3
(10000, 64, 64, 3)
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 71ms/step
../Data/cifar-10-batches-py/data_batch_4
(10000, 64, 64, 3)
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 69ms/step
../Data/cifar-10-batches-py/data_batch_5
(10000, 64, 64, 3)
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 70ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 69ms/step


In [17]:
features_train = np.concatenate(features_train, axis=0)

In [18]:
n_features = features_train.shape[1:]
log_model = Sequential([
    Input(shape=n_features),
    Flatten(),
    Dense(10, activation='softmax')
])
log_model.compile(optimizer=Adam(learning_rate=learning_rate), loss=SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
log_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        81,930 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 81,930 (320.04 KB)

 Trainable params: 81,930 (320.04 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
checkpoint = ModelCheckpoint(
    'model/resnet_model.keras',             # Path where to save the model
    monitor='val_accuracy',       # Metric to monitor
    save_best_only=True,          # Save only the best model
    mode='max',                   # Mode for choosing the best value ('max' since higher accuracy is better)
    verbose=1                     # Verbose output to see when checkpoints are saved
)
log_model.fit(features_train, training_labels, batch_size=BATCH_SIZE, epochs=10, validation_data=(features_test, validation_labels), callbacks=[checkpoint])

Epoch 1/10
1537/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step - accuracy: 0.6835 - loss: 9.2665
Epoch 1: val_accuracy improved from -inf to 0.76200, saving model to model/resnet_model.keras
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 735us/step - accuracy: 0.6841 - loss: 9.2780 - val_accuracy: 0.7620 - val_loss: 9.4282
Epoch 2/10
1540/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step - accuracy: 0.7986 - loss: 7.2866
Epoch 2: val_accuracy improved from 0.76200 to 0.77060, saving model to model/resnet_model.keras
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 614us/step - accuracy: 0.7986 - loss: 7.2979 - val_accuracy: 0.7706 - val_loss: 11.3913
Epoch 3/10
1523/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - accuracy: 0.8272 - loss: 6.4531
Epoch 3: val_accuracy did not improve from 0.77060
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 582us/step - accuracy: 0.8271 - loss: 6.4678 - val_accuracy: 0.7555 - val_loss: 13.2183
Epoch 4/10
1481/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 543us/step - accuracy: 0.8448 - loss: 5.9770
Epoch 4: val_accuracy improved fr

In [20]:
from sklearn.metrics import classification_report
probabilities = log_model.predict(features_test, batch_size=128)

probabilities = np.argmax(probabilities, axis = 1)

print(classification_report(validation_labels, probabilities, target_names= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'], digits=4))

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 582us/step
              precision    recall  f1-score   support

    airplane     0.7610    0.8470    0.8017      1000
  automobile     0.8526    0.8910    0.8714      1000
        bird     0.7417    0.7380    0.7398      1000
         cat     0.6647    0.5750    0.6166      1000
        deer     0.7546    0.6580    0.7030      1000
         dog     0.7453    0.7110    0.7277      1000
        frog     0.7581    0.8870    0.8175      1000
       horse     0.7323    0.8700    0.7952      1000
        ship     0.8734    0.8420    0.8574      1000
       truck     0.9269    0.7730    0.8430      1000

    accuracy                         0.7792     10000
   macro avg     0.7811    0.7792    0.7773     10000
weighted avg     0.7811    0.7792    0.7773     10000

